【任务一    随机森林算法梳理】
1. 集成学习概念
2. 个体学习器概念
3. boosting  bagging
4. 结合策略(平均法，投票法，学习法)
5. 随机森林思想
6. 随机森林的推广
7. 优缺点
8. sklearn参数2
9. 应用场景

【截止时间】
3.1晚22:00前提交博客/Github链接

【参考资料】
《西瓜书》、《统计学习方法》、台大《机器学习技法》、吴恩达《机器学习》等

# 集成学习概念
- 集成学习（ensemble learning）通过构建并结合多个学习器来完成学习任务，有时也被称为`多分类器`（multi-classifier system），基于委员会的学习（committee-based learning）等。它本身不是一个单独的机器学习算法，而是通过构建并结合多个机器学习器来完成学习任务。
- 集成学习的过程：对于训练集数据，我们通过训练若干个个体学习器，通过一定的结合策略，就可以最终形成一个强学习器，以达到博采众长的目的。

# 个体学习器概念
由个体学习器通过一的结合策略，可以形成强学习器。在集成学习整个概念中，由个体学习器之间的差异可以把集成学习分成两种。
- 第一种就是所有的个体学习器都是一个种类的，或者说是同质的。比如都是决策树个体学习器，或者都是神经网络个体学习器。同质集成中的个体学习器亦称为“**基学习器**”。
- 第二种是所有的个体学习器不全是一个种类的，或者说是异质的。比如我们有一个分类问题，对训练集采用支持向量机个体学习器，逻辑回归个体学习器和朴素贝叶斯个体学习器来学习，再通过某种结合策略来确定最终的分类强学习器。异质集成中的个体学习器亦称为“**组件学习器**”。

要获得好的集成，个体学习器应“好而不同”，即个体学习器要有一定的“准确性”和“多样性”。事实上，这两者本身就存在冲突，一般地，准确性很高之后，要增加多样性就要牺牲准确性。如何产生并结合“好而不同”的个体学习器，恰是**集成学习研究的核心**。

# boosting bagging
根据个体学习器的生成方式，将集成学习分成两大类：
- 序列化方法：个体学习器间存在强依赖关系，必须串行生成。（代表：Boosting）
- 并行化方法：个体学习器间不存在强依赖关系，可以同时生成。（代表：Bagging和Random Forest）

## boosting 
Boosting是一族可将弱学习器提升为强学习器的一种框架算法，拥有**系列算法**，如`AdaBoost，GradientBoosting，LogitBoost`等算法。

**工作机制：**
1. 先从出事训练集训练出一个基学习器。
2. 根据基学习器的表现对训练岩本分布进行调整，使得先前基学习器做错的训练样本在后续受到更多关注。
3. 基于调整后的样本分布来训练下一个基学习器。
4. 重复，直至基学习器数目到达事先指定的值T，最终将T个基学习器进行加权结合。

**算法的三个要素：**
1. 函数模型：Boosting的函数模型是**叠加型**的 $$ H(x)=\sum_{t=1}^T {\alpha_t h_t (x)} $$
2. 目标函数：选定某种损失函数作为优化目标 $$ E\{F(x)\}=E\{\sum_{i=1}^k {f_i (x;θ_i)}\} $$
3. 优化算法：贪婪地逐步优化 $$ θ^∗_m=arg\smash{\displaystyle\min_{θ_m}} E\{\sum_{i=1}^{m−1} f_i(x;θ^∗_i)+f_m(x;θ_m)\} $$

**需要解决的问题:**
对于Boosting算法，需要解决两个问题：
1. 如何调整训练集，使得在训练集上**训练**的弱分类器得以进行；
2. 如何将训练得到的各个弱分类器**联合**起来形成强分类器。

**特点：**
- Boosting系列算法的主要区别在于其三要素**选取的函数**不同
- 可以提高任意给定学习算法准确度
- 训练过程为阶梯状，弱分类器**按次序**一一进行训练（实现上可以做到并行），弱分类器的训练集按照某种策略每次都进行一定的转化。最后以一定的方式将弱分类器组合成一个强分类器。
- Boosting中所有的弱分类器可以是**不同类**的分类器


### AdaBoost算法
#### 算法的实现：
1. 若为Adaboost分类，函数模型使用CART分类树；若为Adaboost回归，函数模型使用CART回归树。
2. 损失函数为“指数损失函数”
3. 针对Boosting需要解决的两个问题，AdaBoost算法采用了以下策略：
    * 使用加权后选取的训练数据代替随机选取的训练样本，这样将训练的焦点集中在比较难分的训练数据样本上；
    * 将弱分类器联合起来，使用加权的投票机制代替平均投票机制。让分类效果好的弱分类器具有较大的权重，而分类效果差的分类器具有较小的权重。

#### **特点**
##### 核心思想：
* 针对同一个训练集训练不同的弱分类器，然后把这些弱分类器根据权值集合起来，构造一个更强的最终分类器。
* Adaboost算法是Boosting系列算法之一，强分类器输出结果的是弱分类器输出结果的加权平均。
* Adaboost算法有两个权值，分别为样本权值和弱分类器权值，其主要特征就是在于样本权值的更新和弱分类器权值的更新。
* Adaboost中所有的弱分类器必须是同一种分类器，不能在同一个Adaboost算法的迭代步骤中使用不同的弱分类器。
* 弱分类器可并行实现。

##### 算法的优缺点
* Adaboost的主要优点有：
    1. Adaboost作为分类器时，分类精度很高
    2. 在Adaboost的框架下，可以使用各种回归分类模型来构建弱学习器，非常灵活。
    3. 作为简单的二元分类器时，构造简单，结果可理解。
    4. 不容易发生过拟合
* Adaboost的主要缺点有：
    1. 对异常样本敏感，异常样本在迭代中可能会获得较高的权重，影响最终的强学习器的预测准确性。 


#### 权值更新规则
##### 样本权值更新：
* 对于分类错误的样本，加大其对应的权重；
* 而对于分类正确的样本，降低其权重，这样分错的样本就被突显出来，从而得到一个新的样本分布。

##### 弱分类器权值更新：

* 对于准确率较高的弱分类器，加大其权重；
* 对于准确率较低的弱分类器，减小其权重。

#### 适用范围
1. 用于二分类或多分类的应用场景
2. 用于做分类任务的baseline

#### 算法过程
将样本权值被更新过的新数据集送给下层弱分类器进行训练，最后将每次训练得到的弱分类器根据弱分类器权重融合起来，从而得到强分类器。

1. 给定训练样本集S，其中X和Y分别对应于正例样本和负例样本； T为训练的最大循环次数；
2. 初始化样本权重为1/n ，即为训练样本的初始概率分布；
3. 第一次迭代： 
    * 训练样本的概率分布相当下，训练弱分类器； 
    * 计算弱分类器的错误率； 
    * 选取合适阈值，使得误差最小； 
    * 更新样本权重； 
    * 经T次循环后，得到T个弱分类器，按更新的弱分类器权重叠加，最终得到的强分类器。

### boosting算法小结
该算法要求基学习器能对特定的数据分布进行学习，这可通过**“重赋权法”（re-weighting）**实施，记载训练过程的每一轮中，根据样本分布为每一个训练样本重新赋予一个权重。对无法接受带权样本的基学习算法，则可通过**“重采样法”（re-sampling）**来处理，即在每一轮学习中，根据样本分布对训练集重新采样，再用重新采样的样本对基学习器进行训练。一般而言，这两种做法没有显著的优劣差别。

需要注意的是，boosting算法在训练的每一轮都要检查当前生成的基学习器是否满足基本条件（检查当前基分类器是否是比随机猜测好），一旦条件不满足，则当前基学习器及被抛弃，且学习过程终止。在此情形下，学习轮数达不到T，且学习器的性能不佳。若采用“重采样法”，则可获得“重启动”机会以避免训练过程过早停止。

从**偏差-方差**分解的角度看，boosting主要关注降低偏差，因此boosting能基于繁华性能相当弱的学习器构建出很强的集成。

## bagging
这是并行式集成学习方法最著名的代表。

需解决的问题：为了得到泛化性能强的集成，个体学习器应经可能相互独立（较大的差异），所以用不同的训练集子集来训练基学习器，而这会产生另一个问题：每个基学习器都只用到了一小部分训练数据，不足以进行有效学习。

解决方法：自助采样法：给定包含m个样本的数据集，我们先随机取出一个样本放入采样集中，再把该样本放回初始数据集，使得下次采样是该样本仍有可能被选中，这样，经过m次随机采样操作，我们得到含m个样本的采样集，初始训练集中又的样本在采样集中多次出现，有的从未出现。
照这样，我们可以采样出T个含m个训练样本的采样集，基于每个采样集训练出一个基学习器，再将这些基学习器进行结合。对**预测输出进行结合时**，bagging通常对分类任务进行简单投票法，对回归任务使用简单平均法。

时间复杂度：假定基学习器的时间复杂度是 $O(m)$ ，则bagging的复杂度大致为 $T(O(m) + O(s))$ ,考虑到采样与投票/平均过程的复杂度 $O(s)$ 很小，而T通常是一个不太大的常数，因此，训练一个bagging集成与直接使用基学习算法训练一个学习器的复杂度同阶，这说明bagging是一个很高效的集成学习算法。


**包外估计（out-of-bag estimate）**:另外,由于每个基学习器只使用了初始训练集中约63.2%的样本，剩下的样本可用作验证集来对泛化性能进行“包外估计。保外样本还有其他用途。例如当基学习器式决策树时，可使用保外样本来辅助剪枝，或用于估计决策树中各节点的后验概率以辅助对零训练样本节点的处理；当基学习器时神经网络时，可用包外样本来辅助早期停止以减小过拟合风险。

从**偏差-方差分解的角度看**：bagging主要关注降低方差，因此它在不剪枝决策树、神经网络等一手样本扰动的学习器上效用更为明显。

# 结合策略

假设集成中包含T个基学习器$h_1,h_2,...,h_T$,其中$h_i$在示例$x$上的输出为$h_i(x)$。那么对$h_i$进行结合的常见策略有以下几种：
## 平均法
平均法主要针对回归类任务。

对数值型输出$h_i(x)\in R$,最常见的结合策略是简单平均法（averaging）。加权平均法的权重一般是从训练数据中学习而得，现实任务中的训练样本通常不充分或存在噪声，这将使得学出的权重不完全可靠。因此，实验和应用均显示出，加权平均分未必一定优于朴素的简单平均法。一般而言，在个体学习器的性能相差较大时宜使用加权平均法，而在个体学习器性能相近时宜使用简单平均法。
## 投票法
投票法主要针对分类任务。

对分类任务来说，学习器$h_i$将从类别标记集合${c_1,c_2,...,c_N}$中预测出一个标记，最常见的结合策略是使用投票法。为便于讨论，我们将$h_i$在样本$x$上的预测输出表示为一个N维向量$(h_i^1(x);h_i^2(x);…;h_i^N(x))$，其中$h_i^j(x)$是$h_i$在类别标记$c_j$上的输出。
## 学习法
当训练数据很多时，一种更为强大的结合策略是使用“学习法”，即通过**另一个学习器**来进行结合。Stacking是学习法的典型代表，这里我们把个体学习器称为初级学习器，用于结合的学习器称为次级或者元学习器。

Stacking先从初始训练集训练出初级学习器，然后“生成”一个新数据集用于训练次级学习器。在这个新数据集中，初级学习器的输出被当作样例输入特征，而初始样本的标记仍被当做样例标记。Stacking的算法描述如下图所示，这里假定初始学习器使用不同学习算法产生（例如可同时包含决策树，神经网络算法），即初级集成是异质的。


# 随机森林思想
随机森林（Random Forest，简称RF）是Bagging的一个扩展变体。其在以决策树作为基学习器构建Bagging集成的基础上，进一步在决策树的训练过程中引入了随机属性选择。

具体来说，传统决策树在选择划分属性时是在当前结点的属性集合（假定有d个属性）中选择一个最有属性；而在RF中，对基决策树的每个结点，先从该结点的属性集合中随机选择一个包含k个属性的子集，然后再从这个子集中选择一个最优属性用于划分。这里的参数k控制了随机性的引入程度：若令k=d，则基决策树的构建与传统决策树相同；若令k=1，则是随机选择一个属性用于划分；一般情况下，推荐值$k=log_2d$。

RF简单、容易实现、计算开销小，而令人惊奇的是，它在很多学习任务中展现出强大的性能，被誉为“代表集成学习技术水平的方法”。

RF的收敛性与Bagging相似。随机森林的起始性能往往相对较差，特别是在集成中只包含一个基学习器时，这很容易理解，因为通过引入属性扰动，随机森林中个体学习器的性能往往有所降低。然而，随着个体学习器数目的增加，随机森林通常会收敛到更低的泛化误差。

# 随机森林的推广
